In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'

from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Spark-Exercise") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b83aaaab-a9e1-478d-901f-187693a8d189;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.common

23/03/16 22:47:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Explore dataset FHV

In [36]:
data = spark.read.csv(path="fhv_tripdata_2019-01.csv", sep=",", header=True)
data.filter(data.PUlocationID.isNotNull()).show(10)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00254|2019-01-01 00:33:03|2019-01-01 01:37:24|         140|          52|   null|                B02356|
|              B00254|2019-01-01 00:03:00|2019-01-01 00:34:25|         141|         237|   null|                B00254|
|              B00254|2019-01-01 00:45:48|2019-01-01 01:26:01|         237|         236|   null|                B00254|
|              B00254|2019-01-01 00:37:39|2019-01-01 01:44:59|         162|          85|   null|                B00254|
|              B00254|2019-01-01 00:35:06|2019-01-01 01:30:21|         237|         246|   null|                B00254|
|              B00254|2019-01-01 00:55:2

## Explore dataset Green

In [42]:
dataGreen = spark.read.csv(path="green_tripdata_2019-01.csv", sep=",", header=True)
dataGreen.filter(dataGreen.PULocationID.isNotNull()).show(10)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2018-12-21 15:17:29|  2018-12-21 15:18:57|                 N|         1|         264|         264|              5|          .00|          3|  0.5|    0.

In [43]:
dataGreen.schema

StructType([StructField('VendorID', StringType(), True), StructField('lpep_pickup_datetime', StringType(), True), StructField('lpep_dropoff_datetime', StringType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('passenger_count', StringType(), True), StructField('trip_distance', StringType(), True), StructField('fare_amount', StringType(), True), StructField('extra', StringType(), True), StructField('mta_tax', StringType(), True), StructField('tip_amount', StringType(), True), StructField('tolls_amount', StringType(), True), StructField('ehail_fee', StringType(), True), StructField('improvement_surcharge', StringType(), True), StructField('total_amount', StringType(), True), StructField('payment_type', StringType(), True), StructField('trip_type', StringType(), True), StructField('congestion_surcharge', StringType()

## Reading from Kafka Stream

In [4]:
def parse_ride_from_kafka_message(df_raw, schema):
    """ take a Spark Streaming df and parse value col based on <schema>, return streaming df cols in schema """
    assert df_raw.isStreaming is True, "DataFrame doesn't receive streaming data"

    df = df_raw.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

    # split attributes to nested array in one Column
    col = F.split(df['value'], ', ')

    # expand col to multiple top-level columns
    for idx, field in enumerate(schema):
        df = df.withColumn(field.name, col.getItem(idx).cast(field.dataType))
    return df.select([field.name for field in schema])

def sink_memory(df, query_name, query_template):
    write_query = df \
        .writeStream \
        .queryName(query_name) \
        .format('memory') \
        .start()
    query_str = query_template.format(table_name=query_name)
    query_results = spark.sql(query_str)
    return write_query, query_results

In [44]:
fhv_schema = T.StructType(
    [T.StructField("dispatching_base_num", T.StringType()),
     T.StructField('pickup_datetime', T.TimestampType()),
     T.StructField('dropOff_datetime', T.TimestampType()),
     T.StructField("PUlocationID", T.IntegerType()),
     T.StructField("DOlocationID", T.FloatType()),
     T.StructField("Affiliated_base_number", T.FloatType()),
     ])

green_schema = T.StructType(
    [T.StructField('VendorID', T.StringType(), ), 
    T.StructField('lpep_pickup_datetime', T.TimestampType(), ), 
    T.StructField('lpep_dropoff_datetime', T.TimestampType(), ), 
    T.StructField('store_and_fwd_flag', T.StringType(), ), 
    T.StructField('RatecodeID', T.IntegerType(), ), 
    T.StructField('PULocationID',T.IntegerType(), ), 
    T.StructField('DOLocationID', T.IntegerType(), )
])


In [49]:
# default for startingOffsets is "latest"
df_kafka_raw_fhv = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092,broker:29092") \
    .option("subscribe", "fhv_csv") \
    .option("startingOffsets", "earliest") \
    .option("checkpointLocation", "checkpoint") \
    .load()

# default for startingOffsets is "latest"
df_kafka_raw_green = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092,broker:29092") \
    .option("subscribe", "green_csv") \
    .option("startingOffsets", "earliest") \
    .option("checkpointLocation", "checkpoint") \
    .load()

In [50]:
df_rides_fhv = parse_ride_from_kafka_message(df_raw=df_kafka_raw_fhv, schema=fhv_schema)
df_rides_green = parse_ride_from_kafka_message(df_raw=df_kafka_raw_green, schema=green_schema)

In [51]:
df_rides_fhv.printSchema()
df_rides_green.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: float (nullable = true)
 |-- Affiliated_base_number: float (nullable = true)

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)



In [66]:
query_name = 'q_fhv3'
query_template = 'select PUlocationID from {table_name}'
write_query, df_popular_pulocation_fhv = sink_memory(df=df_rides_fhv, query_name=query_name, query_template=query_template)

query_name = 'q_green3'
query_template = 'select PULocationID from {table_name}'
write_query, df_popular_pulocation_green = sink_memory(df=df_rides_green, query_name=query_name, query_template=query_template)

23/03/16 23:06:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-06fcef72-fbea-446e-8722-32827a81d5b5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/16 23:06:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/03/16 23:06:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1f427527-ebdd-4855-827b-6a15ca1dd03a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/16 23:06:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not support

In [ ]:
df_popular_pulocation_fhv.show()
df_popular_pulocation_green.show()

In [92]:
df_total = df_popular_pulocation_fhv.union(df_popular_pulocation_green)

## Result
Most popular PuLocationID is: 

from pyspark.sql.functions import col
df_total.groupBy('PUlocationID') \
    .count() \
    .sort(col("count").desc()) \
    .show()